## About this notebook

Cross-sectional district level models are run for which features generated by Arpit are used. For every indicator we have around 10 feature files (bin 5-15

Will follow a 5fold cross-validation approach and take the mean of weighted f1-score

Finally all scores will be reported for the best parameters

Not running MLP for this run

In [1]:
import numpy as np                               
import pandas as pd                              
import csv
import time
import keras
import pprint
import copy
import random
import math
import itertools
from operator import itemgetter
from imblearn.over_sampling import SMOTE
import pickle
import os

# mainly for stats models, and tensorflow,pandas version
# related warnings, which should not be ignored ideally
import warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL) 
random.seed(5)
np.random.seed(5)

import src
import specs

# all the specs
from specs.mlSpecs import *
from specs.mlDrillSpecs import *
from specs.ensembleSpecs import *
# from specs.statsSpecs import *

# all the classes and functions for dictionaries
from src.scores import *

from src.models.machineLearning.randomForest import randomForest
from src.models.machineLearning.xgBoost import xgBoost
from src.models.machineLearning.svc import svc
from src.models.machineLearning.adaBoost import adaBoost
from src.models.machineLearning.bagging import bagging
from src.models.machineLearning.gradientBoosting import gradientBoosting
from src.models.machineLearning.knn import knn
from src.models.machineLearning.mlp import mlp
from src.models.machineLearning.ensembleMLP import ensembleMLP

from src.randomizedSearchSpecial import *

#all other functions
from src.dataPreprocessingUtils import *
from src.selectorFunctions import *
from src.mlPrediction import *
from src.combining import *
from src.randomizedSearch import *

Using TensorFlow backend.


In [2]:
# These 3 need to be defined outside in some file which is loaded
scores_dict = {
        "f1_weighted" : {"function" : f1_weighted, "type" : "score",},
        "f1_macro" : {"function" : f1_macro, "type" : "score",},
        "f1_micro" : {"function" : f1_micro, "type" : "score",},
        "accuracy_normal" : {"function" : accuracy_normal, "type" : "score",},
        "accuracy_weighted" : {"function" : accuracy_weighted, "type" : "score",},
        "accuracy_classwise" : {"function" : accuracy_classwise, "type" : "score",},
        "precision_weighted" : {"function" : precision_weighted, "type" : "score",},
        "precision_macro" : {"function" : precision_macro, "type" : "score",},
        "precision_micro" : {"function" : precision_micro, "type" : "score",},
        "recall_weighted" : {"function" : recall_weighted, "type" : "score",},
        "recall_macro" : {"function" : recall_macro, "type" : "score",},
        "recall_micro" : {"function" : recall_micro, "type" : "score",},
#         "roc_auc_weighted" : {"function" : roc_auc_weighted, "type" : "score",},
#         "roc_auc_macro" : {"function" : roc_auc_macro, "type" : "score",},
#         "roc_auc_micro" : {"function" : roc_auc_micro, "type" : "score",},
        "categorical_crossentropy" : {"type" : "error",},
        "accuracy" : {"type" : "score",},
        "acc" : {"type" : "score",}
    }
models_dict = {
#                 "randomForest" : {"class": randomForest , "type": "machine_learning"},
                "xgBoost" : {"class": xgBoost , "type": "machine_learning"},
#                 "svc" : {"class": svc , "type": "machine_learning"},
#                 "bagging" : {"class": bagging , "type": "machine_learning"},
#                 "adaBoost" : {"class": adaBoost , "type": "machine_learning"},
#                 "gradientBoosting" : {"class": gradientBoosting , "type": "machine_learning"},
#                 "knn" : {"class": knn , "type": "machine_learning"},
#                 "mlp" : {"class": mlp , "type": "machine_learning"},
}
special_specs_functions = {
    'randomForest' : multipleSpecialRFSpecs,
    'mlp' : None,
    'xgBoost' : None,
    'svc' : multipleSpecialRFSpecs,
    'adaBoost' : multipleSpecialRFSpecs,
    'gradientBoosting' : multipleSpecialRFSpecs,
    'bagging' : multipleSpecialRFSpecs,
    'knn' : None,
}

In [3]:
print('Models that will be runnning...')
print('-------------------------------')
pprint.pprint(models_dict)

Models that will be runnning...
-------------------------------
{'xgBoost': {'class': <class 'src.models.machineLearning.xgBoost.xgBoost'>,
             'type': 'machine_learning'}}


In [4]:
# Should be a separate function or file # For avoiding bugs and unexpected behaviours # which are hard to find later, no value 
# should be replaced, new values should be # added

for key in specifications_master_dict.keys():
    specifications_master_dict[key].update({
          'n_randomized_search': { 
                        'value_type': 'constant',
                        'values': [200]},
          'test_ratio': { 
                        'value_type': 'constant',
                        'values': [0.0]},
          'y_col': { 
                      'value_type': 'constant',
                      'values': []},          
          'cols_to_drop': { 
                      'value_type': 'constant',
                      'values': [[]]},
           'ensemble': { 
                  'value_type': 'constant',
                  'values': [False]},
           'trail':  { 
                  'value_type': 'categories',
                  'values': [0]},
          'n_cols_dropped': { 
                      'value_type': 'constant',
                      'values': [0]},
          'current_date_col': { 
                      'value_type': 'constant',
                      'values': []},
          'no_steps_ahead': { 
                      'value_type': 'constant',
                      'values': [0]},
          'top_n_models_drill': { 
                             'value_type': 'constant',
                             'values': [5]},
          'top_n_models_pred': { 
                             'value_type': 'constant',
                             'values': [5]},
          'apply_smote': { 
                             'value_type': 'constant',
                             'values': [True]},      # can be True or False
          'time_cols': {    
                             'value_type': 'constant',
                             'values': [[]]},
          'n_drill_search': { 
                     'value_type': 'constant',
                     'values': [[100]]},
          'cross_validation': { 
                     'value_type': 'constant',
                     'values': [['KFold']]},     # can take values - KFold, LeaveOneOut, train_val_holdout
                                                 # LeavePOut - in case of KFold & LeavePOut, we should have 
                                                 # additional parameter of k and p
         'kFold_splits': { 
                             'value_type': 'categories',
                             'values': [5]}, 
         'pFold_splits': { 
                             'value_type': 'categories',
                             'values': [3,4,5,10,15,20]}, 
         'feature_cols': { 
                             'value_type': 'constant',
                             'values': []},},
    )
# pprint.pprint(specifications_master_dict)
print('Done Updating the Common Specs')

Done Updating the Common Specs


In [5]:
# pprint.pprint(specifications_master_dict)

In [6]:
# def apply_smote(data, feature_cols, target):
#     '''
#     Input:
#     data - the original dataframe
#     feature_cols - the feature columns (list of columns)
#     target - the target column (string value)
#     '''
#     sm = SMOTE(random_state=42)
#     features, targets = sm.fit_resample(data[feature_cols],data[target])
#     feature_df = pd.DataFrame(features, columns=feature_cols)
#     target_df = pd.DataFrame(targets, columns=[target])
#     output = pd.concat([feature_df, target_df], axis=1)
#     # Shuffling dataset
#     output = output.sample(frac=1).reset_index(drop=True)
#     return output

## Data

In [7]:
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/GroundTruth_ChangeClassifier.csv")

In [8]:
ground_truth.head()

,census_code,MSW_change,MSW_2001,BF_change,BF_2001,CHH_change,CHH_2001,EMP_change,EMP_2001,MSL_change,MSL_2001,FC_change,FC_2001,ASSET_change,ASSET_2001,Formal_EMP_2001,LIT_2001
0,1,0,1,0,2,0,1,0,1,1,2,0,1,0,1,2,1
1,2,0,3,0,2,1,1,0,1,0,3,0,2,1,1,2,1
2,3,0,1,0,2,1,1,0,3,0,3,0,3,1,1,3,1
3,4,1,1,1,2,0,1,0,3,0,3,0,2,0,1,3,1
4,5,0,1,0,1,0,1,0,2,1,2,0,1,0,1,2,1


In [9]:
folder = "/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data"
for files in os.listdir(folder):
    
    df = pd.read_csv(os.path.join(folder, files))
    data = ground_truth.merge(df, on='census_code', how='left')
    data.dropna(inplace=True)
    
    target = files.split('_')[0] + '_change'
    print('#### Running for : ',target)
    
    feature_cols = ['Formal_EMP_2001','LIT_2001','predictions_2001', 'predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009','predictions_2011']
    
    drop_cols = []
    for col in data.columns:
        if col in feature_cols:
            continue
        elif col == target:
            continue
        drop_cols.append(col)
        
#     print(type(drop_cols))
    
    print(target, drop_cols, feature_cols)
    for key in specifications_master_dict.keys():
               specifications_master_dict[key].update({
                     'y_col': {
                                 'value_type': 'constant',
                                 'values': [target]},
                     'current_date_col': {
                                 'value_type': 'constant',
                                 'values': drop_cols},
                    'feature_cols': {
                                        'value_type': 'constant',
                                        'values': [feature_cols]},},
               )
    print('-------------------------------------')
    print('-------------------------------------') 
    model_specs_vs_score = callModels(models_dict, scores_dict, specifications_master_dict, data, False)
    top_scores_specs = topSpecs(model_specs_vs_score, scores_dict,get_train_score=True)
    print('------------------------')
    print(top_scores_specs['xgBoost']['val_scores'])
    print(top_scores_specs['xgBoost']['train_scores'])
    pickle.dump(top_scores_specs, open(target + '_CC_IncludeFormalEMP&LIT_XGB.pkl', 'wb'))
    print('------------------------')

#### Running for :  CHH_change
CHH_change ['census_code', 'MSW_change', 'MSW_2001', 'BF_change', 'BF_2001', 'CHH_2001', 'EMP_change', 'EMP_2001', 'MSL_change', 'MSL_2001', 'FC_change', 'FC_2001', 'ASSET_change', 'ASSET_2001'] ['Formal_EMP_2001', 'LIT_2001', 'predictions_2001', 'predictions_2003', 'predictions_2005', 'predictions_2007', 'predictions_2009', 'predictions_2011']
-------------------------------------
-------------------------------------
Running model  xgBoost
####### n_randomized_search [200]
    run 0   CV Type ['KFold'] 5
    run 1   CV Type ['KFold'] 5
    run 2   CV Type ['KFold'] 5
    run 3   CV Type ['KFold'] 5
    run 4   CV Type ['KFold'] 5
    run 5   CV Type ['KFold'] 5
    run 6   CV Type ['KFold'] 5
    run 7   CV Type ['KFold'] 5
    run 8   CV Type ['KFold'] 5
    run 9   CV Type ['KFold'] 5
    run 10   CV Type ['KFold'] 5
    run 11   CV Type ['KFold'] 5
    run 12   CV Type ['KFold'] 5
    run 13   CV Type ['KFold'] 5
    run 14   CV Type ['KFold'] 5
    

KeyboardInterrupt: 